# Imports

In [32]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras import layers, models

# load images and labels from a directory

In [33]:
# Function to load images and their corresponding labels from the directory
def load_data(directory):
    images = []  # To store image data
    labels = []  # To store corresponding labels
    class_names = sorted(os.listdir(directory))  # Get class names in sorted order
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for file_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, file_name)
                img = load_img(img_path, target_size=img_size)  # Load image and resize to specified size
                img = img_to_array(img)  # Convert image to array format
                images.append(img)  # Append image to the list
                labels.append(label)  # Append label (numeric class index) to the list
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

# Define directories for training and testing datasets
train_dir = 'C:\\Users\\dovy4\\Desktop\\jupyter test\\1000'
test_dir = 'C:\\Users\\dovy4\\Desktop\\jupyter test\\test_100'

# Define image size (32x32 pixels)
img_size = (32, 32)

# Load and shuffle training data
train_images, train_labels, class_names = load_data(train_dir)
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

# Load and shuffle test data
test_images, test_labels, _ = load_data(test_dir)
test_images, test_labels = shuffle(test_images, test_labels, random_state=42)

# Normalize the data

In [34]:
# Normalize the image data to range [0, 1] by dividing by 255
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# One-hot encode labels

In [35]:
# Convert labels to one-hot encoding format
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(class_names))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(class_names))

# Define the CNN model for 32x32 images

for this part more layers were initially added however this caused overfitting in the training process and a more simplified version actually gave us better results.

In [36]:
# Define the CNN model architecture
model = models.Sequential([
    # First convolutional layer with 32 filters, each of size 3x3
    # Activation function: ReLU (Rectified Linear Unit)
    # Input shape specified for the first layer: 32x32 image with 3 color channels (RGB)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
    
    # MaxPooling layer to reduce the spatial dimensions (downsampling)
    # Pool size of 2x2, reduces the size of the feature maps by half
    layers.MaxPooling2D((2, 2)),
    
    # Second convolutional layer with 64 filters, each of size 3x3
    # Activation function: ReLU
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    
    # MaxPooling layer to further reduce the spatial dimensions
    layers.MaxPooling2D((2, 2)),
    
    # Third convolutional layer with 128 filters, each of size 3x3
    # Activation function: ReLU
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    
    # MaxPooling layer to reduce spatial dimensions for the last time
    layers.MaxPooling2D((2, 2)),
    
    # Flatten layer to convert 2D matrices into a 1D vector (required before feeding into fully connected layers)
    layers.Flatten(),
    
    # Fully connected (Dense) layer with 256 units
    # Activation function: ReLU
    layers.Dense(256, activation='relu'),
    
    # Dropout layer to prevent overfitting by randomly setting a fraction of input units to 0 during training
    # Dropout rate: 50% (0.5)
    layers.Dropout(0.5),
    
    # Output layer with units equal to the number of classes (class_names)
    # Activation function: Softmax (for multi-class classification)
    layers.Dense(len(class_names), activation='softmax')
])

# Compile the model with Adam optimizer, categorical crossentropy loss function, and accuracy metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model

In [37]:
# Define a callback to reduce learning rate when the validation loss has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5)

# Train the model using the training data, with 20 epochs and a validation split of 20%
# The reduce_lr callback is used to adjust the learning rate during training
history = model.fit(
    train_images, train_labels,
    epochs=20,
    validation_split=0.2,
    batch_size=32,
    callbacks=[reduce_lr]
)

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.1951 - loss: 2.1345 - val_accuracy: 0.3715 - val_loss: 1.7231 - learning_rate: 0.0010
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.3878 - loss: 1.6471 - val_accuracy: 0.4615 - val_loss: 1.4778 - learning_rate: 0.0010
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4692 - loss: 1.4624 - val_accuracy: 0.5085 - val_loss: 1.3917 - learning_rate: 0.0010
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5164 - loss: 1.3305 - val_accuracy: 0.5315 - val_loss: 1.3081 - learning_rate: 0.0010
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5518 - loss: 1.2270 - val_accuracy: 0.5595 - val_loss: 1.2465 - learning_rate: 0.0010
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5865 - loss: 1.1334 - val_accuracy: 0.5910 - val_loss: 1.1788 - learning_rate: 0.0010
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6352 - loss: 1.

# Evaluate the model

In [38]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6852 - loss: 1.0735
Test accuracy: 0.6729999780654907


# Save the model

In [ ]:
model.save('cnn_model.h5')